In [6]:
import pandas as pd
import pickle
songs = pd.read_pickle('songsWeeklyRanks.pkl')

In [20]:
import unicodedata

def getStringWithoutSpecialAccent(string):
    return ''.join((c for c in unicodedata.normalize('NFD', string) if unicodedata.category(c) != 'Mn'))
getStringWithoutSpecialAccent("Michael Bublé".lower())

'michael buble'

In [7]:
songs["SpotifyID"] = None

In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = "2db050f400f245738228ef384306044b"
secret = "6309c1f0455b43428d2cf142e4130a64"

client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [16]:
def getSongID(trackName, trackArtist, trackResults):
    for track in trackResults['tracks']['items']:
        if (track['name'].lower() == trackName.lower()) & (track['artists'][0]['name'].lower() == trackArtist.lower()):
            return track['id']
    return None

def getAlbumID(albumName, albumArtist, searchResult):
    for album in searchResult['albums']['items']:
        getStringWithoutSpecialAccent(album['name'].lower())
        getStringWithoutSpecialAccent(albumArtist.lower())
        
        getStringWithoutSpecialAccent(album['artists'][0]['name'].lower())
        getStringWithoutSpecialAccent(albumArtist.lower())
        if (album['name'].lower() == albumName.lower()) & (album['artists'][0]['name'].lower() == albumArtist.lower()):
            return album['id']
    return None

def getAlbumSongs(albumArtist, albumTracksResults):
    albumSongDF = pd.DataFrame(columns=["Title","Artist", "SpotifyID"])
    for songs in albumTracksResults['items']:
        newRow = pd.Series(data={"Title": songs['name'],"Artist": albumArtist, "SpotifyID": songs['id']})
        albumSongDF = albumSongDF.append(newRow, ignore_index=True)
    return albumSongDF

In [10]:
##create two df
songsToDeleteDF = pd.DataFrame(columns=["Title","Artist", "SpotifyID"])
albumSongToAddDF = pd.DataFrame(columns=["Date","Title","Artist","Rank", "SpotifyID"])

spotifyIDForSongsCache = {} #stores spotify ids for songs that you already queried
songsDSForAlbumCache = {} #stores DataFrame of Songs (Title,Artist,SpotifyID) from an Album

#iterate through all song rows and find SpotifyID
for index, row in songs.head(2).iterrows(): 
    songNameAndArtist = row['Title'] + "-" + row['Artist'] #used for key values on caches
    if spotifyIDForSongsCache.get(songNameAndArtist) is not None: #if it exists in cache, use that instead
        #print(songNameAndArtist + " Used Song Cache")
        songID = spotifyIDForSongsCache[songNameAndArtist]
        songs.SpotifyID.iloc[index] = songID
        continue
    if songsDSForAlbumCache.get(songNameAndArtist) is not None: #if it exists in cache, use that instead
        #print(songNameAndArtist + " Used Album Cache")
        songsFromAlbum = songsDSForAlbumCache[songNameAndArtist]
        songsFromAlbum["Date"] = row["Date"]
        songsFromAlbum["Rank"] = row["Rank"]
        albumSongToAddDF = pd.concat([albumSongToAddDF,songsFromAlbum], ignore_index=True)
        continue
    
    #get songID from Spotify Search API
    trackResults = sp.search(q=row['Title'], type='track', market= 'US', limit=10,offset=0)
    songID = getSongID(row['Title'], row['Artist'], trackResults)
    #If songID is not found, we assume it is an album and search for its albumID
    if  songID == None:
        print("Finding Album")
        #Delete the row from songs DF that does not have spotifyID since they are an album
        songsToDeleteDF = songsToDeleteDF.append(row, ignore_index=True)
        searchResult = sp.search(q=row['Title'], type='album', market= 'US', limit=10,offset=0)
        albumID = getAlbumID(row['Title'], row['Artist'], searchResult)
        #If we find an albumID, we get all the songs from that album and add it to the DF

        if albumID != None:
            print("Finding songs")
            albumTracksResults = sp.album_tracks(albumID)
            songsFromAlbum = getAlbumSongs(row['Artist'],albumTracksResults)
            #cache
            keyToStore = songNameAndArtist
            songsDSForAlbumCache[keyToStore] = songsFromAlbum
            #add date and rank
            songsFromAlbum["Date"] = row["Date"]
            songsFromAlbum["Rank"] = row["Rank"]
            
            albumSongToAddDF = pd.concat([albumSongToAddDF,songsFromAlbum], ignore_index=True)
    else:
        #Add the spotifyID to an existing song
        #cache 
        keyToStore = songNameAndArtist
        spotifyIDForSongsCache[keyToStore] = songID
        
        songs.SpotifyID.iloc[index] = songID


Finding Album


In [17]:
getAlbumID(row['Title'], row['Artist'], sp.search(q="Christmas", type='album', market= 'US', limit=10,offset=0))